In [ ]:
import pandas as pd
import geopandas as gpd
import json

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
 
# WWF ECOREGION MAP DATA


# read in wwf map data
map = pd.read_json(
    '/home/muskrat/Documents/eco_data/wwf_map_data/wwf_terr_ecos.json')
# ej = pd.read_json(
#     '/media/muskrat/060A9B8E0A9B78FF/map_final/eco_map.geojson')
# print(ej['features'][0])


In [ ]:

 
# WWF ECOREGION MAP DATA

# unpack json file and convert to dataframe


def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        tmp = pd.DataFrame((d for idx, d in df[column].iteritems()))
        ret = pd.concat([df.drop(column, axis=1), tmp], axis=1)
    else:
        tmp = pd.DataFrame((d for idx, d in
                            df[column].iteritems())).fillna(fillna)
        ret = pd.concat([df.drop(column, axis=1), tmp], axis=1)
    return ret


unpacked_features = unpack(map, 'features', 0)

unpacked_properties = unpack( unpacked_features, 'properties', 0)

# unpacked_properties.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)
# unpacked_properties.reset_index(drop=True, inplace=True)

# ej3 = unpack(unpacked_properties, 'geometry', 0)

# ej3.dropna(subset=['unique_id'], inplace=True)
# ej3.reset_index(drop=True, inplace=True)

# # df = pd.DataFrame(ej3.coordinates[0])

# # df1 = df[0][0]

# selected = []

# for i in range(len(unpacked_properties)):
#     # df2 = pd.DataFrame(ej3.coordinates[i])
#     # df3 = df2[0][0]
#     df3 = unpacked_properties.geometry[i]['coordinates'][-1][-1]
#     if len(df3) > 2:
#         selected.append(df3[-1])
#     else:
#         selected.append(df3)
# # merged_data = ej3['coordinates'][0][0][0]

# import math

# desk = []

# for i in range(len(selected)):
#     d = selected[i]

#     deesk = []
#     for j in range(len(d)):
#         factor = 10.0 ** 3
#         t = math.trunc(d[j] * factor) / factor
#         deesk.append(t)
#     desk.append(deesk)

# unpacked_properties['point'] = desk
# unpacked_properties.rename(columns={'point': 'coordinates'}, inplace=True)

# ecoregions = unpacked_properties[['unique_id', 'name', 'TYPE', 'coordinates']]

# ej3.iloc[[0], []] = merged_data


# create subset of dataframe just to check data
# ej3 = unpacked_properties[['OBJECTID', 'ECO_NAME', 'ECO_NUM', 'ECO_ID', 'eco_code']]

# rename eco_code column
unpacked_properties.rename(columns={'eco_code': 'ECOREGION_CODE'}, inplace=True)
unpacked_properties.drop(unpacked_properties.columns[0], axis=1, inplace=True)
unpacked_properties.drop(unpacked_properties.columns[0], axis=1, inplace=True)

# unpacked_properties.insert(0, 'type', 'Feature')

# create subset of dataframe
ecoregions_subset = unpacked_properties[['unique_id', 'ECOREGION_CODE']]

merged_data = pd.merge(unpacked_properties, ecoregions_subset, on='ECOREGION_CODE', how='left')


data_dict = merged_data.to_dict(orient='records')

# data_dict = unpacked_properties.to_dict(orient='records')
# add properties wrapper key to values

properties_key = "properties"

properties_dictionary = [{properties_key: property} for property in data_dict]

# convert to json
json_data = json.dumps(properties_dictionary)

properties_df = pd.DataFrame(properties_dictionary)

unpacked_features.drop(unpacked_features.columns[0], axis=1, inplace=True)
unpacked_features.drop(unpacked_features.columns[1], axis=1, inplace=True)

unpacked_features.insert(0, 'type', 'Feature')
# unpacked_features.insert(0, 'type', 'Feature')

# join dataframes

map_data = pd.concat([unpacked_features, properties_df], axis=1, join='inner')
map_json = map_data.to_json(orient='records', force_ascii=False)
# eco = ecoregions.to_json(orient='records', force_ascii=False)

# map_json = properties_df.to_json(orient='records', force_ascii=False)
repr(map_json)

# add start and end tag to json

map_final = '{"type":"FeatureCollection", "features": ' + map_json + '}'


In [ ]:


 
# WWF ECOREGION MAP DATA

# write to json file
file = open("dataset.json", "w")
file.write(map_final)
file.close


In [ ]:

 
ecomap = gpd.read_file(
    '/media/muskrat/060A9B8E0A9B78FF/maptest/terr_map.geojson')


In [ ]:

 
marinemap = gpd.read_file(
    '/media/muskrat/060A9B8E0A9B78FF/maptest/marine_map.geojson')


In [ ]:

 
ecomap['name'] = ecomap['ECO_NAME']

ecomap["unique_id"] = ecomap["unique_id"].astype("float32")

ecomap['unique_id'] = pd.to_numeric(ecomap['unique_id'], downcast='integer')

ecomap["unique_id"] = ecomap["unique_id"].convert_dtypes()

ecomap['unique_id'] = ecomap['unique_id'].astype(str)

ecomap.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

marinemap['name'] = ''

marinemap.loc[marinemap['TYPE'] == 'PPOW', 'name'] = marinemap['PROVINC']

marinemap.loc[marinemap['TYPE'] == 'MEOW', 'name'] = marinemap['ECOREGION']

marinemap = marinemap[['unique_id', 'name', 'TYPE', 'geometry']]

ecomap['TYPE'] = 'TEOW'

ecomap = ecomap[['unique_id', 'name', 'TYPE', 'geometry']]



In [ ]:

 

marinemap.to_file("marine_map.geojson", driver='GeoJSON')

ecomap.to_file("terr_map.geojson", driver='GeoJSON')

In [ ]:
ecomap = gpd.read_file(
    '/media/muskrat/060A9B8E0A9B78FF/eco_data/wwf_terr_map_data/terr_map.geojson')


In [ ]:

 
marinemap = gpd.read_file(
    '/media/muskrat/060A9B8E0A9B78FF/eco_data/marine_map_data/marine_map.geojson')
# marinemap = gpd.read_file(
#     '/media/muskrat/060A9B8E0A9B78FF/map_final/eco_map.geojson')


In [ ]:

 


ranger = [*range(826, 826+232)]
ranger2 = [*range(1058, 1058+37)]
ranger3 = ranger2 + ranger
marine_ids = [str(x) for x in ranger3]

marinemap.insert(0, 'unique_id', marine_ids)

 

marinemap.to_file("marine_map.geojson", driver='GeoJSON')


In [ ]:

 


ecomap['name'] = ecomap['ECO_NAME']

ecomap["unique_id"] = ecomap["unique_id"].astype("float32")

ecomap.drop(ecomap[ecomap['unique_id'].isna()].index, inplace=True)

ecomap["unique_id"] = ecomap["unique_id"].astype("int32")

ecomap['unique_id'] = ecomap['unique_id'].apply(str)

ecomap.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

marinemap['name'] = ''

marinemap.loc[marinemap['TYPE'] == 'PPOW', 'name'] = marinemap['PROVINC']

marinemap.loc[marinemap['TYPE'] == 'MEOW', 'name'] = marinemap['ECOREGION']

econames = pd.concat([ecomap[['unique_id', 'name']], marinemap[[
                     'unique_id', 'name']]], axis=0, keys=['unique_id', 'name'])


# econames.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

econames.reset_index(drop=True, inplace=True)


In [ ]:

 
ecokeys = econames.to_json(orient='records', force_ascii=False)
repr(ecokeys)


In [ ]:

 

file = open("ecoregion_keys.json", "w")
file.write(ecokeys)
file.close